# Backtracking Visualisation for the AntiSlop Sampler

This notebook demonstrates the AntiSlop sampler by adding delays when slop phrases are detected & replaced, as well as some debug output.

https://github.com/sam-paech/antislop-sampler

### Update 2024-10-05

- Switch to probs from logits for the cached values, so that down/upregulation works as expected.
- Use model.generate for multiple tokens (not just 1 at a time) with StoppingCondition criteria. This is much faster.
- Add json validation including long-range checks for unintended unescaped quotes in strings.

In [1]:
#!pip install transformers ipywidgets IPython

In [2]:
import os
import json
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from IPython.display import display, HTML
from ipywidgets import Output
from src.antislop_generate import AntiSlopSampler, chat_antislop, generate_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# These are a mix of gpt-slop found in various online lists, plus a much larger set of automatically derived over-represented words in a GPT-generated dataset.
# See slopcalc.ipynb for the code to generate a more complete list.
slop_phrase_prob_adjustments = [['kaleidoscope', 0.5], ['symphony', 0.5], ['testament to', 0.5], ['elara', 0.5], ['moth to a flame', 0.5], ['canvas', 0.5], ['eyes glinted', 0.5], ['camaraderie', 0.5], ['humble abode', 0.5], ['cold and calculating', 0.5], ['eyes never leaving', 0.5], ['tapestry', 0.5], ['barely above a whisper', 0.5], ['body and soul', 0.5], ['orchestra', 0.5], ['depths', 0.5], ['a dance of', 0.5], ['chuckles darkly', 0.5], ['maybe, just maybe', 0.5], ['maybe that was enough', 0.5], ['with a mixture of', 0.5], ['air was filled with anticipation', 0.5], ['cacophony', 0.5], ['bore silent witness to', 0.5], ['eyes sparkling with mischief', 0.5], ['was only just beginning', 0.5], ['practiced ease', 0.5], ['ready for the challenges', 0.5], ['only just getting started', 0.5], ['once upon a time', 0.5], ['nestled deep within', 0.5], ['ethereal beauty', 0.5], ['life would never be the same again.', 0.5], ["it's important to remember", 0.5], ['for what seemed like an eternity', 0.5], ['feel a sense of pride and accomplishment', 0.5], ['little did he know', 0.5], ['ball is in your court', 0.5], ['game is on', 0.5], ['choice is yours', 0.5], ['feels like an electric shock', 0.5], ['threatens to consume', 0.5], ['meticulous', 0.5], ['meticulously', 0.5], ['navigating', 0.5], ['complexities', 0.5], ['realm', 0.5], ['understanding', 0.5], ['dive into', 0.5], ['shall', 0.5], ['tailored', 0.5], ['towards', 0.5], ['underpins', 0.5], ['everchanging', 0.5], ['ever-evolving', 0.5], ['world of', 0.5], ['not only', 0.5], ['alright', 0.5], ['embark', 0.5], ['journey', 0.5], ["today's digital age", 0.5], ['game changer', 0.5], ['designed to enhance', 0.5], ['it is advisable', 0.5], ['daunting', 0.5], ['when it comes to', 0.5], ['in the realm of', 0.5], ['amongst', 0.5], ['unlock the secrets', 0.5], ['unveil the secrets', 0.5], ['and robust', 0.5], ['diving', 0.5], ['elevate', 0.5], ['unleash', 0.5], ['cutting-edge', 0.5], ['rapidly', 0.5], ['expanding', 0.5], ['mastering', 0.5], ['excels', 0.5], ['harness', 0.5], ["it's important to note", 0.5], ['delve into', 0.5], ['bustling', 0.5], ['in summary', 0.5], ['remember that', 0.5], ['take a dive into', 0.5], ['landscape', 0.5], ['in the world of', 0.5], ['vibrant', 0.5], ['metropolis', 0.5], ['firstly', 0.5], ['moreover', 0.5], ['crucial', 0.5], ['to consider', 0.5], ['essential', 0.5], ['there are a few considerations', 0.5], ['ensure', 0.5], ["it's essential to", 0.5], ['furthermore', 0.5], ['vital', 0.5], ['keen', 0.5], ['fancy', 0.5], ['as a professional', 0.5], ['however', 0.5], ['therefore', 0.5], ['additionally', 0.5], ['specifically', 0.5], ['generally', 0.5], ['consequently', 0.5], ['importantly', 0.5], ['indeed', 0.5], ['thus', 0.5], ['alternatively', 0.5], ['notably', 0.5], ['as well as', 0.5], ['despite', 0.5], ['essentially', 0.5], ['even though', 0.5], ['in contrast', 0.5], ['in order to', 0.5], ['due to', 0.5], ['even if', 0.5], ['given that', 0.5], ['arguably', 0.5], ['you may want to', 0.5], ['on the other hand', 0.5], ['as previously mentioned', 0.5], ["it's worth noting that", 0.5], ['to summarize', 0.5], ['ultimately', 0.5], ['to put it simply', 0.5], ["in today's digital era", 0.5], ['reverberate', 0.5], ['enhance', 0.5], ['emphasize', 0.5], ['revolutionize', 0.5], ['foster', 0.5], ['remnant', 0.5], ['subsequently', 0.5], ['nestled', 0.5], ['labyrinth', 0.5], ['gossamer', 0.5], ['enigma', 0.5], ['whispering', 0.5], ['sights unseen', 0.5], ['sounds unheard', 0.5], ['indelible', 0.5], ['my friend', 0.5], ['in conclusion', 0.5], ['technopolis', 0.5], ['was soft and gentle', 0.5], ['shivers down', 0.5], ['shivers up', 0.5], ['leaving trails of fire', 0.5], ['ministrations', 0.5], ['audible pop', 0.5], ['rivulets of', 0.5], ['despite herself', 0.5], ['reckless abandon', 0.5], ['torn between', 0.5], ['fiery red hair', 0.5], ['long lashes', 0.5], ['propriety be damned', 0.5], ['world narrows', 0.5], ['chestnut eyes', 0.5], ['cheeks flaming', 0.5], ['cheeks hollowing', 0.5], ['understandingly', 0.5], ['paperbound', 0.5], ['hesitantly', 0.5], ['piqued', 0.5], ['delved', 0.5], ['curveballs', 0.5], ['marveled', 0.5], ['inclusivity', 0.5], ['birdwatcher', 0.5], ['newfound', 0.5031423922762257], ['marveling', 0.5055622891781474], ["hiroshi's", 0.506870969939047], ['greentech', 0.5095092042816856], ['thoughtfully', 0.510153898156777], ['intently', 0.5153227374075411], ['birdwatching', 0.5157928537951464], ['amidst', 0.5161190296674488], ['cherishing', 0.5165772000484282], ['attentively', 0.5169695157301188], ['interjected', 0.5208671011920856], ['serendipitous', 0.5219535186850968], ["marianne's", 0.5220118279910801], ["maya's", 0.5229467776607973], ['excitedly', 0.5235248665614571], ['steepled', 0.5235772300889154], ['engrossed', 0.5236764398055735], ['fostering', 0.5259281627970829], ['brainstormed', 0.5274863713437], ['furrowed', 0.5280860997212533], ['nodded', 0.528640180937889], ['contemplatively', 0.5293698584415747], ['jotted', 0.5300819077932343], ["mia's", 0.5311706933553655]];
slop_phrase_prob_adjustments = dict(slop_phrase_prob_adjustments)

if os.path.exists('slop_phrase_prob_adjustments.json'):
    with open('slop_phrase_prob_adjustments.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:500])

### Test anti-slop sampling

In [8]:
# Example prompt for the model
prompt = "Once upon a time, in a bustling city of Technopolis, there lived a weaver named Elara."

# Define the messages for a chat scenario (for example purposes)
messages = [
    {"role": "user", "content": prompt}
]

# Display the output widgets
prompt_output = Output()
inference_output = Output()
debug_output = Output()

display(HTML("<h2>Prompt</h2>"))
display(prompt_output)
display(HTML("<h2>Inference Output</h2>"))
display(inference_output)
display(HTML("<h2>Debug Information</h2>"))
display(debug_output)

# Initialize display (if using in Jupyter)
with prompt_output:
    prompt_output.clear_output(wait=True)
    display(HTML(f"<div style='white-space: pre-wrap;'>{prompt}</div>"))

if True:
    VALIDATE_JASON_STRINGS = True
    # Call the chat_antislop to generate the story with the given messages
    output_tokens = chat_antislop(
        model=model,
        tokenizer=tokenizer,
        messages=messages,
        max_new_tokens=1000,        
        temperature=1,
        min_p=0.1,
        slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
        adjustment_strength=100,
        device=device,
        streaming=False,
        slow_debug=True,  # Enable slow debugging
        output_every_n_tokens=3,
        debug_delay=1,  # Set delay for debug output
        inference_output=inference_output,  # Visualization of the text output
        enforce_json=False,
        debug_output=debug_output  # Visualization of the debug information
    )

    inference = tokenizer.decode(output_tokens, skip_special_tokens=True)

Output()

Output()

Output()



In the heart of the city, where skyscrapers pierced the sky and neon lights danced across the streets, there lived a young and talented weaver named Eliana. But she was not the same as the famous weaver of the same name, who had lived in the city for centuries.

While her ancestor, the great Eliana, had been renowned for her exquisite tapestries that told stories of the ancient lore and myths of the land, Eliana's own story was one of hard work and determination.

She lived in a small, cluttered workshop in the lower levels of the city, surrounded by rows of looms that hummed with the soft murmur of threads being woven into life. The air was thick with the scent of wool and the faint tang of magic, a subtle aroma that only the most skilled weavers could detect.

Eliana's fingers moved deftly, the threads flying through the looms as she worked on her latest project – a stunning velvet cloak for the city's most esteemed patron, the Queen herself. She was driven by a fierce ambition, de

## Evaluate json constraint enforcement

Note: This evaluation will fail output that contains multiple separate json elements (like json-lines format). The enforce_json flag does not enforce this; it only enforces json validity within a json block.

As such, it's suggested to use a model that will at least follow the output format for this eval to work as intended (i.e. a stronger model than the example Llama-3.2-1B used above)

In [4]:
from tqdm import tqdm

prompt = 'Write a short scene of dialogue between two characters, Jill and Steve. Output in json with this format: {"scene": "<dialogue goes here>"}'

# Define the messages for a chat scenario (for example purposes)
messages = [
    {"role": "user", "content": prompt}
]

if False:
    inference_output = Output()
    debug_output = Output()
    display(HTML("<h2>Inference Output</h2>"))
    display(inference_output)
    display(HTML("<h2>Debug Information</h2>"))
    display(debug_output)

    n_iterations = 20

    valid_count = 0
    for i in tqdm(range(n_iterations)):
        output_tokens = []    

        output_tokens = chat_antislop(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            max_new_tokens=800,        
            temperature=1,
            min_p=0.1,
            slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
            adjustment_strength=5.0,
            device=device,
            streaming=False,
            slow_debug=False,  # Enable slow debugging
            output_every_n_tokens=1,  # Update every 5 tokens
            debug_delay=1,  # Set delay for debug output
            inference_output=inference_output,  # Visualization of the text output
            enforce_json=True,
            debug_output=debug_output  # Visualization of the debug information
        )

        inference = tokenizer.decode(output_tokens, skip_special_tokens=True)
        lpos = inference.find('{')
        rpos = inference.rfind('}')
        if lpos >= 0 and rpos > 0:
            inference = inference[lpos:rpos+1]
        try:
            parsed = json.loads(inference)
            valid_count += 1
        except Exception as e:
            print([inference])
            print(e)
            pass

        print('json successfully parsed:', valid_count, 'of', n_iterations)
        print('')


Output()

Output()

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:01<00:25,  1.35s/it]

['{"scene": "Jill and Steve sat at the small café, sipping their coffee and discussing their plans for the day.",}']
Expecting property name enclosed in double quotes: line 1 column 112 (char 111)
json successfully parsed: 0 of 20



...

json successfully parsed: 16 of 20



 95%|█████████▌| 19/20 [03:19<00:11, 11.48s/it]

json successfully parsed: 17 of 20



100%|██████████| 20/20 [03:25<00:00, 10.29s/it]

json successfully parsed: 18 of 20

